Assuming your data is streamed from Cloud storage (e.g. Google Cloud storage), you might want to build a script to continuously uploading data from there. Let's check it out

Check what we have in Google Storage and create the list of tasks

In [ ]:
from google.cloud import storage as google_storage

BUCKET_NAME = 'my-bucket'  # specify your bucket name here
GOOGLE_APPLICATION_CREDENTIALS = 'my-service-account-credentials.json'  # specify your GCS credentials

google_client = google_storage.Client()
bucket = google_client.get_bucket(BUCKET_NAME)
tasks = []
for filename in bucket.list_blobs():
    tasks.append({'image': f'gs://{BUCKET_NAME}/{filename}'})

Now create a Label Studio project...

In [ ]:
from label_studio_sdk import Client
LABEL_STUDIO_URL = 'http://localhost:8000'
API_KEY = 'd6f8a2622d39e9d89ff0dfef1a80ad877f4ee9e3'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

project = ls.start_project(
    title='Image Annotation Project from SDK',
    label_config='''
    <View>
        <Image name="image" value="$image"/>
        <RectangleLabels name="objects" toName="image">
            <Choice value="Airplane"/>
            <Choice value="Car"/>
        </RectangleLabels>
    </View>
    '''
)

... and connect Google Cloud Storage to it

In [ ]:
project.connect_google_import_storage(
    bucket=BUCKET_NAME,
    google_application_credentials=GOOGLE_APPLICATION_CREDENTIALS
)

Now you can upload your private GCS links - they gonna be presigned automatically!

In [ ]:
project.import_tasks(tasks)